In [ ]:
!pip install datasketch
from datasketch import MinHash, MinHashLSH
import pandas as pd


     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     ------------------------- ------------ 41.0/60.8 kB 653.6 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 537.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/89.2 kB ? eta -:--:--
   --------------------------- ------------ 61.4/89.2 kB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 89.2/89.2 kB 1.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/44.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/44.5 MB 5.1 MB/s eta 0:00:09
   ---------------------------------------- 0.1/44.5 MB 5.1 MB/s eta 0:00:09
   ---------------------------------------- 0.2/44.5 MB 1.4 MB/s eta 0:00:32
   ---------------------------------------- 0.2/44.5 MB 1.5 MB/s eta 0:00:30
   ---------------------------------------- 0.2/44.5 MB 1.5 MB/s eta 0:00:30
   -----------

Min-Hash

In [ ]:
df = pd.read_csv(r"C:\Users\Sneha Gautam\Downloads\NLP_assignment 1\Cleaned_dataset\cleaned_Jang_all_articles.csv")
df = df[df['Content'].notna()]
original_length = len(df)
print("Original DataFrame length:", original_length)

lsh = MinHashLSH(threshold=0.8, num_perm=128)

def create_minhash(text):
    minhash = MinHash(num_perm=128)
    for word in text.split():
        minhash.update(word.encode('utf8'))
    return minhash

for index, row in df.iterrows():
    minhash = create_minhash(row['Content'])  # Use 'content' column
    lsh.insert(f"doc_{index}", minhash)

duplicate_indices = set()

for index, row in df.iterrows():
    if f"doc_{index}" not in duplicate_indices:
        minhash = create_minhash(row['Content'])
        result = lsh.query(minhash)

        for similar_doc in result:
            similar_index = int(similar_doc.split('_')[1])
            if similar_index != index:
                duplicate_indices.add(similar_index)

dedup_df = df[~df.index.isin(duplicate_indices)]

deduplicated_length = len(dedup_df)
print("Deduplicated DataFrame length:", deduplicated_length)

dedup_df.to_csv(r"C:\Users\Sneha Gautam\Downloads\deduplicated_cleaned_Jang_all_articles.csv", index=False)

Sim-Hash

In [ ]:
import hashlib
import numpy as np
import pandas as pd
from simhash import Simhash, SimhashIndex

df = pd.read_csv(r"C:\Users\Sneha Gautam\Downloads\NLP_assignment 1\Cleaned_dataset\cleaned_Jang_all_articles.csv")
df = df[df['Content'].notna()]
original_length = len(df)
print("Original DataFrame length:", original_length)

def create_simhash(text):
    return Simhash(text.split())

data = [(f"doc_{index}", create_simhash(row['Content'])) for index, row in df.iterrows()]

index = SimhashIndex(data, k=3)  # k is the threshold, 3 means 3 bits of difference

duplicate_indices = set()

for i, row in df.iterrows():
    if f"doc_{i}" not in duplicate_indices:
        simhash_obj = create_simhash(row['Content'])
        similar_docs = index.get_near_dups(simhash_obj)

        for similar_doc in similar_docs:
            similar_index = int(similar_doc.split('_')[1])
            if similar_index != i:
                duplicate_indices.add(similar_index)

dedup_df = df[~df.index.isin(duplicate_indices)]

deduplicated_length = len(dedup_df)
print("Deduplicated DataFrame length:", deduplicated_length)

dedup_df.to_csv(r"C:\Users\Sneha Gautam\Downloads\deduplicated_cleaned_Jang_all_articles_simhash.csv", index=False)


Cosine Similarity

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

df = pd.read_csv(r"C:\Users\Sneha Gautam\Downloads\NLP_assignment 1\Cleaned_dataset\cleaned_Jang_all_articles.csv")
df = df[df['Content'].notna()]
original_length = len(df)
print("Original DataFrame length:", original_length)

vectorizer = TfidfVectorizer().fit_transform(df['Content'])
tfidf_matrix = vectorizer.toarray()

cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

threshold = 0.8

duplicate_indices = set()

for i in range(len(df)):
    if i not in duplicate_indices:
        for j in range(i + 1, len(df)):
            if cosine_sim_matrix[i, j] > threshold:
                duplicate_indices.add(j)

dedup_df = df[~df.index.isin(duplicate_indices)]

deduplicated_length = len(dedup_df)
print("Deduplicated DataFrame length:", deduplicated_length)

dedup_df.to_csv(r"C:\Users\Sneha Gautam\Downloads\deduplicated_cleaned_Jang_all_articles_cosine_similarity.csv", index=False)
